# Exercice 1: I think they’re sending us Signals

## Parameters

### Tom

In [36]:
Q1_ops = ['send', 'send', 'send', 'send', 'send', 'send', 'send', 'send', 'send', 'send', 'send', 'send', 'send', 'send', 'send', 'send', 'send', 'send', 'send', 'send', 'send', 15, 13, 4, 8, 21, 6, 12, 2, 10]
Q1_k = 31

### Manon

In [48]:
Q1_ops = ['send', 'send', 'send', 'send', 'send', 'send', 'send', 'send', 'send', 'send', 'send', 'send', 'send', 'send', 'send', 'send', 'send', 'send', 'send', 'send', 'send', 'send', 'send', 16, 17, 8, 10, 1, 22, 21]
Q1_k = 32

## Ratchet

### Ratchet behaviour

We'll try to implement the behaviour of the ratchet to see the state known after the sequence of operations for both Alice and Bob.

In [49]:
def send(i,state,vv=False):
    assert i >= 1 and i <= 70
    # when A sends a message, they create and immediately
    # delete MK_i-1. CK_i-1 is deleted and
    # CK_i is the only kept key.
    new_CK = 'CK_{}'.format(i)
    state[i-1]=(None,None)
    state[i]=(new_CK,None)
    if vv:
        print("Sending message {}|state[{}]={}|state[{}]={}".format(i,i-1,state[i-1],i,state[i]))
    return state,i+1

In [50]:
def recv(i,max_val,state,vv=False):
    assert i >= 1 and i <= 70
    if i < max_val: # we know we already computed the MK_i
        # we can just consume it
        state[i-1]=(None,None)
        if vv:
            print("Receiving message {} (i<max)|state[{}]={}|max was {}".format(i,i-1,state[i-1],max_val))
    elif i > max_val:
        # we'll compute all and delete all CK_j, j in {0,...,i-1}
        # while keeping the MK_j but for j=i-1
        # we take care of the first call when max_val=-1
        log_string="Receiving message {} (i>max)|\n".format(i)
        for j in range(max(0,max_val),i): # exclusive upper bound
            MK_j = 'MK_{}'.format(j)
            state[j]=(None,MK_j)
            log_string+="\tstate[{}]={}\n".format(j,state[j])
        # we delete MK_i-1 and CK_i-1 as MK_i-1 is consumed
        state[i-1]=(None,None)
        log_string+="\tstate[{}]={}\n".format(i-1,state[i-1])
        new_CK = 'CK_{}'.format(i)
        # we keep CK_i for next time
        state[i]=(new_CK,None)
        log_string+="\tstate[{}]={}\n\tmax was {}".format(i,state[i],max_val)
        if vv:
            print(log_string)
    else:
        print("Unexpected arguments, i:{}==max:{}".format(i,max_val))
        return None,None,None
    return state,max(max_val,i) # returns new state and max index seen

In [51]:
def ratchet_from_ops(ops,vv=False):
    # we recall that we can maximally send 70 messages
    # (sending message i requires using MK from state i-1)
    A_state = [(None,None) for i in range(71)] # initialize null state
    A_i = 1
    B_state = [(None,None) for i in range(71)]
    B_max = -1
    for opi in ops:
        if opi=='send': # we track Alice's state
            A_state,A_i = send(A_i,A_state,vv)
        else: # we track Bob's state
            assert opi >= 0
            B_state,B_max = recv(opi,B_max,B_state,vv)
    return {"Alice":A_state,"Bob":B_state}

From those we can take a look at the state of A and B after the operations sequence. Alice's sequence is first and Bob's second.

In [52]:
states=ratchet_from_ops(Q1_ops,vv=False) 
print(states)

{'Alice': [(None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), ('CK_23', None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None,

## Q1_1 - derive messages

### Alice state's exposure
From Alice's state exposure, we can see from the ratchet behaviour that only the last $CK$ is exposed and no $MK$. The ratchet purpose is fulfilled here, there's no way to go back if Alice correctly deleted $RK_{init}$ which is the only piece missing to regenerate a previous $CK$ (With $KDF_{RK}(RK_{init},DH_{new})$ as $DH_{new}$ is leaked).

Indeed, we can see how many messages an attacker could derive by observing the amount of leaked $MK$ as those are used to decrypt the messages.

In [53]:
Q1a_1 = set()
for index,state in enumerate(states["Alice"]):
    if state[1] is not None:
        Q1a_1.add(index+1)
print("{}" if len(Q1a_1) == 0 else Q1_a1)

{}


### Bob's state exposure
From Bob's state exposure, the ratchet state of Bob shows that all messages that have not yet been decrypted but have index $<max$ have a stored key by Bob $MK$ ! Therefore, all messages sent by Alice but not yet received by Bob can be decrypted with the information leaked by Bob's state.

Noting that $MK_i$ can be used to decrypt message $i+1$, we could decrypt all messages $m_{i+1}$ for which the $MK_i$ hasn't been computed yet.

We here observe a consequence of the tradeoff between functionnality (random access to decryption) and security (strict ratchet principle)

In [54]:
Q1b_1 = set()
for index,state in enumerate(states["Bob"]):
    if state[1] is not None:
        Q1b_1.add(index+1)
print("{}" if len(Q1b_1) == 0 else Q1b_1)

{2, 3, 4, 5, 6, 7, 9, 11, 12, 13, 14, 15, 18, 19, 20}


## Q1_2 - forge messages

### Alice's state exposure
In this setting, Alice's state exposure yields $CK_{max}$ which can be used to generate $MK_{max},...,MK_{69}$ that are the key values used to decrypt messages $m_{max+1},...,m_{70}$ !

Therefore, all those subsequent messages can be easily forged by an attacker without Bob being able to distinguish who sent it. Bob will just advance their ratchet correspondingly to their $recv$ calls.

In [55]:
Q1a_2 = set()
can_forge = False
for index,state in enumerate(states["Alice"]):
    if can_forge:
        Q1a_2.add(index)
    if state[0] is not None:
        # order of instruction important:
        # can forge only messages that has not yet been received
        # namely message i+1 when we have CK_i (used to generate MK_i that decrypts msg i+1)
        can_forge = True
print("{}" if len(Q1a_2) == 0 else Q1a_2)

{24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70}


### Bob's state exposure
In this setting, we have the same exposure of $CK_{max}$ which allows us to decrypt at least as many messages as in Alice's state exposure. Moreover, as Bob built some unused $MK_i$ messages keys but not used it yet as not having yet called `recv(i)` for those, an attacker could use those keys to spoof the messages $i$ sent by Alice and make Bob believe that the true messages Alice sent is the attacker's.

Therefore, all messages after $max$ **and** messages not yet received can be forged.

In [56]:
Q1b_2 = set()
can_forge = False
for index,state in enumerate(states["Bob"]):
    if can_forge:
        Q1b_2.add(index)
    if state[0] is not None:
        # order of instruction important:
        # can forge only messages that has not yet been received
        # namely message i+1 when we have CK_i (used to generate MK_i that decrypts msg i+1)
        can_forge = True
    if state[1] is not None:
        # has access to a not yet used MK_i, making us able to forge msg i+1
        Q1b_2.add(index+1)
print("{}" if len(Q1b_2) == 0 else Q1b_2)

{2, 3, 4, 5, 6, 7, 9, 11, 12, 13, 14, 15, 18, 19, 20, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70}


## Q1_3 - forge with signatures

### Alice's state exposure
In this setting, Alice's state being exposed allow us to forge exactly the same messages as the scheme without signature. Indeed, the signing key being also exposed does not increase the complexity of the attack.

In [57]:
Q1a_3 = set()
can_forge = False
for index,state in enumerate(states["Alice"]):
    if can_forge:
        Q1a_3.add(index)
    if state[0] is not None:
        # order of instruction important:
        # can forge only messages that has not yet been received
        # namely message i+1 when we have CK_i (used to generate MK_i that decrypts msg i+1)
        can_forge = True
print("{}" if len(Q1a_3) == 0 else Q1a_3)

{24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70}


### Bob's state exposure
However here, as the attacker does not have access to Alice's signing key, there's no way to forge any message that will be accepted by Bob's receive method.

In [58]:
Q1b_3 = set()
print("{}" if len(Q1b_3) == 0 else Q1_b3)

{}


## Q1_4 - maximize state

In [59]:
Q1_B = [Q1_k]
Q1_B

[32]